# **라이브러리 로드**

In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 데이터 전처리 패키지
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# 모델 패키지
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential

# 모델 평가 패키지
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")


# **1. 순환 신경망(Recurrent Neural Network, RNN)**

- 루프(loop)를 가진 신경망의 한 종류

- 시퀀스의 원소를 순회하면서 지금까지 처리한 정보를 상태(state)에 저장

<img src="https://miro.medium.com/max/627/1*go8PHsPNbbV6qRiwpUQ5BQ.png">

## **1.1. RNN 모델 실습 : IMDB 데이터 with RNN(이진분류)**

### **IMDB 데이터 셋** ###
- 영화 리뷰 데이터
- 영화 리뷰, 긍정/부정 레이블로 구성(이진 분류)
- 텍스트 분석 작업에서 주로 활용
- 50000개의 리뷰 데이터 (Train 25000, Test 25000)


### **데이터 로드**

In [5]:
data = pd.read_csv("../data/IMDB Dataset.csv")

In [6]:
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


### **데이터 전처리**

#### **Pad sequences**
- 가변 길이 시퀀스를 동일한 길이로 맞추기 위해 패딩을 추가하는 함수
- 입력 시퀀스를 리스트로 받아 가장 긴 시퀀스에 맞춰 나머지 시퀀스 뒤에 패딩 값을 추가

In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)  # 'sentiment' 열의 값을 'positive' -> 1, 'negative' -> 0으로 변환
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)  # 데이터를 80:20 비율로 훈련/테스트 세트로 분할

tokenizer = Tokenizer(num_words=5000)  # 최대 5000개의 단어를 사용해 텍스트를 토큰화하는 Tokenizer 생성
tokenizer.fit_on_texts(train_data["review"])  # 훈련 데이터의 'review' 텍스트로 Tokenizer에 단어 사전 생성

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)  # 훈련 데이터를 시퀀스로 변환하고, 길이를 200으로 패딩
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)    # 테스트 데이터를 시퀀스로 변환하고, 길이를 200으로 패딩

Y_train = train_data["sentiment"]  # 훈련 데이터의 'sentiment' 열을 레이블로 설정
Y_test = test_data["sentiment"]    # 테스트 데이터의 'sentiment' 열을 레이블로 설정

### **모델 구성**

In [ ]:
model = Sequential()
model.add(Embedding(5000, 128))
model.add(SimpleRNN(128, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss = 'binary_crossentropy',
              metrics=['acc'])
model.summary()

### **모델 학습**

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs = 10,
                    batch_size = 128,
                    validation_split = 0.2)

### **학습과정 시각화**

In [ ]:
loss = history.history['loss']  # 훈련 데이터의 손실 값 추적
val_loss = history.history['val_loss']  # 검증 데이터의 손실 값 추적
acc = history.history['acc']  # 훈련 데이터의 정확도 추적
val_acc = history.history['val_acc']  # 검증 데이터의 정확도 추적

epochs = range(1, len(loss)+1)  # 에포크 범위 생성

# 손실 값 시각화
plt.plot(epochs, loss, 'b--', label='training loss')  # 훈련 손실 그래프 (파란 점선)
plt.plot(epochs, val_loss, 'r:', label='validation loss')  # 검증 손실 그래프 (빨간 점선)
plt.grid()  # 격자 표시
plt.legend()  # 범례 표시

# 새로운 Figure 생성 후 정확도 시각화
plt.figure()
plt.plot(epochs, acc, 'b--', label='training accuracy')  # 훈련 정확도 그래프 (파란 점선)
plt.plot(epochs, val_acc, 'r:', label='validation accuracy')  # 검증 정확도 그래프 (빨간 점선)
plt.grid()  # 격자 표시
plt.legend()  # 범례 표시

# 그래프 출력
plt.show()

### **모델 평가**

In [ ]:
model.evaluate(X_test, Y_test)

## **1.2. RNN 모델 실습 : Reuters 데이터 with RNN(다중분류)**

### **Reuters**

- IMDB와 유사한 텍스트 데이터셋
- 1986년 로이터에서 공개한 짧은 뉴스 기사 및 토픽의 집합
- 46개의 상호 배타적인 토픽으로 이루어진 데이터셋(다중 분류)


### **데이터 로드**

In [ ]:
# reuters.npz 파일에서 데이터 로드
data = np.load('./data/reuters.npz', allow_pickle=True)

# 데이터로부터 x, y변수 추출
x = data['x']
y = data['y']

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 데이터 차원 확인
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# 데이터를 동일한 길이로 패딩 처리
x_train = pad_sequences(x_train, maxlen=100, padding='post')
x_test = pad_sequences(x_test, maxlen=100, padding='post')

In [ ]:
# 원 핫 인코딩
y_train = to_categorical(y_train, num_classes=46)  # 46은 클래스 수
y_test = to_categorical(y_test, num_classes=46)

### **모델 구성**

In [ ]:
model2 = Sequential()
model2.add(Embedding(10000, 500))
model2.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(46, activation='softmax'))


model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['acc'])

model2.summary()

### **모델 학습**

In [ ]:
history2 = model2.fit(x_train, y_train,
                    batch_size = 128, epochs = 10,
                    validation_split=0.20)

### **학습과정 시각화**

In [ ]:
loss = history2.history['loss']
val_loss = history2.history['val_loss']
acc = history2.history['acc']
val_acc = history2.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'b--', label = 'training loss')
plt.plot(epochs, val_loss, 'r:', label = 'validation loss')
plt.grid()
plt.legend()

plt.figure()
plt.plot(epochs, acc, 'b--', label = 'training accuracy')
plt.plot(epochs, val_acc, 'r:', label = 'validation accuracy')
plt.grid()
plt.legend()

plt.show()

### **모델 평가**

In [ ]:
model2.evaluate(X_test, Y_test)

#### **SimpleRNN의 한계점**

- SimpleRNN은 실전에 사용하기엔 너무 단순

- SimpleRNN은 이론적으로 시간 $t$ 에서 이전의 모든 타임스텝의 정보를 유지할 수 있지만, 실제로는 긴 시간에 걸친 의존성은 학습할 수 없음

- 그래디언트 소실 문제(vanishing gradient problem)
  - 이를 방지하기 위해 **LSTM** 등장



***

# **2. LSTM(Long Short-Term Memory)**
- 장단기 메모리 알고리즘

- 나중을 위해 정보를 저장함으로써 오래된 시그널이 점차 소실되는 것을 막아줌

  <img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png">

  <sub>출처: https://colah.github.io/posts/2015-08-Understanding-LSTMs/</sub>

## **2.1. LSTM 모델 실습 : IMDB 데이터(이진분류)**

### **데이터 로드**

In [ ]:
data = pd.read_csv("./data/IMDB Dataset.csv")

### **데이터 전처리**

In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)  # 'sentiment' 열의 값을 'positive' -> 1, 'negative' -> 0으로 변환
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)  # 데이터를 80:20 비율로 훈련/테스트 세트로 분할

tokenizer = Tokenizer(num_words=5000)  # 최대 5000개의 단어를 사용해 텍스트를 토큰화하는 Tokenizer 생성
tokenizer.fit_on_texts(train_data["review"])  # 훈련 데이터의 'review' 텍스트로 Tokenizer에 단어 사전 생성

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)  # 훈련 데이터를 시퀀스로 변환하고, 길이를 200으로 패딩
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)    # 테스트 데이터를 시퀀스로 변환하고, 길이를 200으로 패딩

Y_train = train_data["sentiment"]  # 훈련 데이터의 'sentiment' 열을 레이블로 설정
Y_test = test_data["sentiment"]    # 테스트 데이터의 'sentiment' 열을 레이블로 설정

### **모델 구성**

In [ ]:
model3 = Sequential()
model3.add(Embedding(5000, 128))
model3.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='rmsprop',
              loss = 'binary_crossentropy',
              metrics=['acc'])
model3.summary()

### **모델 학습**

In [ ]:
history3 = model3.fit(X_train, Y_train,
                    epochs = 10,
                    batch_size =128,
                    validation_split=0.2)

### **시각화**

In [ ]:
loss = history3.history['loss']
val_loss = history3.history['val_loss']
acc = history3.history['acc']
val_acc = history3.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'b--', label = 'training loss')
plt.plot(epochs, val_loss, 'r:', label = 'validation loss')
plt.grid()
plt.legend()

plt.figure()
plt.plot(epochs, acc, 'b--', label = 'training accuracy')
plt.plot(epochs, val_acc, 'r:', label = 'validation accuracy')
plt.grid()
plt.legend()

plt.show()

### **모델 평가**

In [ ]:
model3.evaluate(X_test, Y_test)

***

## **2.2. LSTM 모델 실습 : Reuters 데이터 (다중분류)**

### 데이터셋 로드

In [ ]:
# reuters.npz 파일에서 데이터 로드
data = np.load('./data/reuters.npz', allow_pickle=True)

# 데이터로부터 x, y변수 추출
x = data['x']
y = data['y']

# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 데이터 차원 확인
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

### **데이터 전처리 및 확인**

In [ ]:
# 데이터를 동일한 길이로 패딩 처리
x_train = pad_sequences(x_train, maxlen=100, padding='post')
x_test = pad_sequences(x_test, maxlen=100, padding='post')

In [ ]:
# 원 핫 인코딩
y_train = to_categorical(y_train, num_classes=46)  # 46은 클래스 수
y_test = to_categorical(y_test, num_classes=46)

### **모델 구성**

In [ ]:
model2 = Sequential()
model2.add(Embedding(10000, 500))
model2.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(46, activation='softmax'))


model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['acc'])

model2.summary()

### **모델 학습**

In [ ]:
history2 = model2.fit(x_train, y_train,
                    batch_size = 128, epochs = 10,
                    validation_split=0.20)

### **학습과정 시각화**

In [ ]:
loss = history2.history['loss']
val_loss = history2.history['val_loss']
acc = history2.history['acc']
val_acc = history2.history['val_acc']

epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'b--', label = 'training loss')
plt.plot(epochs, val_loss, 'r:', label = 'validation loss')
plt.grid()
plt.legend()

plt.figure()
plt.plot(epochs, acc, 'b--', label = 'training accuracy')
plt.plot(epochs, val_acc, 'r:', label = 'validation accuracy')
plt.grid()
plt.legend()

plt.show()

### **모델 평가**

In [ ]:
model2.evaluate(X_test, Y_test)

***

# **3. 시계열 데이터 셋을 활용한 RNN / LSTM**

#### **Airline-passengers dataset**
- 1949년 1월 ~ 1960년 12월(144개월) 사이의 월간 비행기 이용객 수

## **3.1. 시계열 데이터 셋을 활용한 RNN**

### **데이터 로드 및 확인**

In [ ]:
dataset = pd.read_csv('./data/airline-passengers.csv', usecols=[1], engine='python')
plt.plot(dataset)
plt.show()

### **데이터 로드 및 전처리**

In [ ]:
# 난수생성기 시드 설정
np.random.seed(2020)

# 데이터 불러오기
dataset = dataset.values
dataset = dataset.astype('float32')

# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# 데이터 분리
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# 배열의 값을 데이터셋 행렬로 변환
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# X=t 와 Y=t+1 로 변환
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# 배열을 [samples, time steps, features] 형태로 변환
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

### **모델 구성**

In [ ]:
def build_model(n):
    model = Sequential()
    model.add(SimpleRNN(units = 32, activation='tanh', input_shape=(1,look_back)))
    model.add(Dense(1))
    
    model.compile(optimizer='adam',
                  loss='mse')
    
    return model

In [ ]:
model = build_model(10)
model.summary()

### **모델 학습**

In [ ]:
model.fit(trainX, trainY,
          epochs=100, batch_size=1, verbose=2)

### **모델 예측**

In [ ]:
# 예측 수행
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# 예측 반전
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# MSE 산출
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

### **예측 결과 시각화**

In [ ]:
# 훈련 예측 값 plot을 위한 shift 수행
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# 테스트 예측 값 plot을 위한 shift 수행
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

## **3.2. 실습 : 시계열 데이터 셋을 활용한 LSTM**

### **데이터 로드 및 확인**

In [ ]:
dataset = pd.read_csv('./data/airline-passengers.csv', usecols=[1], engine='python')
plt.plot(dataset)
plt.show()

### **데이터 로드 및 전처리**

In [ ]:
# 난수생성기 시드 설정
np.random.seed(2020)

# 데이터 불러오기
dataset = dataset.values
dataset = dataset.astype('float32')

# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# 데이터 분리
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [ ]:
# 배열의 값을 데이터셋 행렬로 변환하는 함수
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# X=t 와 Y=t+1 로 변환
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# 배열을 [samples, time steps, features] 형태로 변환
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

### **모델 구성**

In [ ]:
def build_model(n):
    model = Sequential()
    model.add(SimpleRNN(units = 32, activation='tanh', input_shape=(1,look_back)))
    model.add(Dense(1))
    
    model.compile(optimizer='adam',
                  loss='mse')
    
    return model

In [ ]:
model = build_model(10)
model.summary()

### **모델 학습**

In [ ]:
model.fit(trainX, trainY,
          epochs=100, batch_size=1, verbose=2)

### **모델 예측**

In [ ]:
# 예측 수행
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# 예측 반전
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# MSE 산출
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

### **예측 결과 시각화**

In [ ]:
# 훈련 예측 값 plot을 위한 shift 수행
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# 테스트 예측 값 plot을 위한 shift 수행
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()